契約期間２&フル間取りでXGBoostのクロスバリデーションを行う。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib
import xgboost
from sklearn.model_selection import KFold

###訓練データの読み込み
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange = pd.read_csv('room_arrange.csv')
contract_span = pd.read_csv('contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('reg_rent.csv')
floor_scores = pd.read_csv('floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')
addresses = pd.read_csv('addresses.csv')
room_arrange = pd.read_csv('room_arrange.csv')
buildings = pd.read_csv('buildings.csv')

rent = pd.read_csv('rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_contract_span2 = pd.read_csv('test_contract_span2.csv')
test_reg_rent = pd.read_csv('test_reg_rent.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('test_capital_floor_scores2.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_buildings = pd.read_csv('test_buildings.csv')

Duplicate key in file PosixPath('/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc'), line 258 ('font.family : IPAexGothic')
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


チューニングを行う。

In [4]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

def objective(trial):
    
    xgb_reg = xgboost.XGBRegressor(
    #目的関数の指定 初期値も二乗誤差です
                       objective='reg:linear',
                       #学習のラウンド数 early_stoppingを利用するので多めに指定
                       n_estimators=50000,
                       #boosterに何を用いるか 初期値もgbtreeです
                       booster='gbtree',
                       #学習率
                       learning_rate=0.05,
                       #木の最大深さ
                       max_depth=trial.suggest_int('max_depth', 1, 10),
                       #シード値
                       random_state=0,
                        #正則化
                       reg_alfa=trial.suggest_float('reg_alfa', 0.0000001, 0.0001, log=True),

                       sub_sample=trial.suggest_float('sub_sample', 0.01, 0.9, log=True),

                       colsample_bytree=trial.suggest_float('colsample_bytree', 0.01, 0.9, log=True),

                       min_child_weight=trial.suggest_float('min_child_weight', 0.01, 10.0, log=True),
    )

    xgb_reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10)
    y_pred_valid = xgb_reg.predict(X_valid)

    loss = mean_squared_error(y_valid, y_pred_valid, squared=False)
    return loss

In [5]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)
study.best_params

[I 2022-09-11 19:35:00,696] A new study created in memory with name: no-name-022e336a-daa0-430a-9ea4-25e361a3c12d


[19:35:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[19:35:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "reg_alfa", "sub_sample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:133361.18197
[1]	validation_0-rmse:128557.85334
[2]	validation_0-rmse:123888.73198
[3]	validation_0-rmse:119165.61808
[4]	validation_0-rmse:114909.17270
[5]	validation_0-rmse:111277.62356
[6]	validation_0-rmse:107570.49423
[7]	validation_0-rmse:104403.28986
[8]	validation_0-rmse:101468.31058
[9]

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[35]	validation_0-rmse:61968.81106
[36]	validation_0-rmse:61446.06189
[37]	validation_0-rmse:61122.21143
[38]	validation_0-rmse:60710.57102
[39]	validation_0-rmse:60452.54393
[40]	validation_0-rmse:60239.90601
[41]	validation_0-rmse:59903.41899
[42]	validation_0-rmse:59711.62022
[43]	validation_0-rmse:59354.72912
[44]	validation_0-rmse:58098.31262
[45]	validation_0-rmse:57969.68529
[46]	validation_0-rmse:57529.52096
[47]	validation_0-rmse:57423.15208
[48]	validation_0-rmse:57293.08848
[49]	validation_0-rmse:57153.65112
[50]	validation_0-rmse:57016.08097
[51]	validation_0-rmse:56763.49232
[52]	validation_0-rmse:56403.06133
[53]	validation_0-rmse:56279.63135
[54]	validation_0-rmse:55799.46794
[55]	validation_0-rmse:55551.11555
[56]	validation_0-rmse:55121.07850
[57]	validation_0-rmse:54764.28448
[58]	validation_0-rmse:54673.37726
[59]	validation_0-rmse:54608.28032
[60]	validation_0-rmse:54574.93778
[61]	validation_0-rmse:54262.24224
[62]	validation_0-rmse:54020.33611
[63]	validation_0-rm

[I 2022-09-11 19:35:06,167] Trial 0 finished with value: 34818.31823078172 and parameters: {'max_depth': 6, 'reg_alfa': 1.3981961408994034e-05, 'sub_sample': 0.1506419097075089, 'colsample_bytree': 0.11610044354231804, 'min_child_weight': 0.18662266976517972}. Best is trial 0 with value: 34818.31823078172.


...

[19:44:20] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[19:44:20] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "reg_alfa", "sub_sample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:133361.18197
[1]	validation_0-rmse:128557.85334
[2]	validation_0-rmse:123888.73198
[3]	validation_0-rmse:119165.61808
[4]	validation_0-rmse:114909.17270
[5]	validation_0-rmse:111277.62356
[6]	validation_0-rmse:107570.49423
[7]	validation_0-rmse:104445.92740
[8]	validation_0-rmse:101512.66327
[9]

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[37]	validation_0-rmse:61307.27700
[38]	validation_0-rmse:60897.54047
[39]	validation_0-rmse:60640.60030
[40]	validation_0-rmse:60442.79297
[41]	validation_0-rmse:60108.08365
[42]	validation_0-rmse:59945.35985
[43]	validation_0-rmse:59596.12104
[44]	validation_0-rmse:58354.29063
[45]	validation_0-rmse:58226.33899
[46]	validation_0-rmse:57784.14340
[47]	validation_0-rmse:57678.33130
[48]	validation_0-rmse:57548.94327
[49]	validation_0-rmse:57410.29114
[50]	validation_0-rmse:57327.95771
[51]	validation_0-rmse:57074.71604
[52]	validation_0-rmse:56711.91497
[53]	validation_0-rmse:56589.12526
[54]	validation_0-rmse:56107.39047
[55]	validation_0-rmse:55912.62522
[56]	validation_0-rmse:55480.87419
[57]	validation_0-rmse:55122.13392
[58]	validation_0-rmse:55072.54849
[59]	validation_0-rmse:55008.12566
[60]	validation_0-rmse:54975.22051
[61]	validation_0-rmse:54660.31927
[62]	validation_0-rmse:54414.40766
[63]	validation_0-rmse:54070.83383
[64]	validation_0-rmse:54010.06384
[65]	validation_0-rm

[I 2022-09-11 19:44:26,001] Trial 39 finished with value: 36078.882392985615 and parameters: {'max_depth': 5, 'reg_alfa': 4.811094460982537e-07, 'sub_sample': 0.031410456573584364, 'colsample_bytree': 0.012983836916378706, 'min_child_weight': 0.20102491014919222}. Best is trial 1 with value: 23093.509985916466.


{'max_depth': 7,
 'reg_alfa': 2.054762512591133e-06,
 'sub_sample': 0.5530198332197337,
 'colsample_bytree': 0.7642417009494488,
 'min_child_weight': 0.14135935551752302}

チューニングしたパラメータを行う。

In [7]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K","S"]]], axis=1)
y_train = rent

# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K","S"]]], axis=1)

cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    xgb_reg = xgboost.XGBRegressor(
        #目的関数の指定 初期値も二乗誤差です
                        objective='reg:linear',
                        #学習のラウンド数 early_stoppingを利用するので多めに指定
                        n_estimators=50000,
                        #boosterに何を用いるか 初期値もgbtreeです
                        booster='gbtree',
                        #学習率
                        learning_rate=0.05,
                        #木の最大深さ
                        max_depth=7,
                        #シード値
                        random_state=0,
                            #正則化
                        reg_alfa=2.054762512591133e-06,

                        sub_sample=0.5530198332197337,

                        colsample_bytree=0.7642417009494488,

                        min_child_weight=0.14135935551752302
    )
    xgb_reg.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], early_stopping_rounds=10)
y_pred = xgb_reg.predict(X_test)


[19:49:48] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:48] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "reg_alfa", "sub_sample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:132565.58842
[1]	validation_0-rmse:126650.14447
[2]	validation_0-rmse:121107.24070
[3]	validation_0-rmse:115799.37748
[4]	validation_0-rmse:110981.84982
[5]	validation_0-rmse:106400.85105
[6]	validation_0-rmse:101849.13745
[7]	validation_0-rmse:97539.63943
[8]	validation_0-rmse:93418.79629
[9]	v

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[15]	validation_0-rmse:71180.92067
[16]	validation_0-rmse:68569.92215
[17]	validation_0-rmse:66096.38690
[18]	validation_0-rmse:63790.77657
[19]	validation_0-rmse:61599.28034
[20]	validation_0-rmse:59680.57700
[21]	validation_0-rmse:57763.18435
[22]	validation_0-rmse:56211.07039
[23]	validation_0-rmse:54569.78817
[24]	validation_0-rmse:53131.65252
[25]	validation_0-rmse:51676.11348
[26]	validation_0-rmse:50233.53663
[27]	validation_0-rmse:49022.52469
[28]	validation_0-rmse:47810.04225
[29]	validation_0-rmse:46786.28632
[30]	validation_0-rmse:45661.30819
[31]	validation_0-rmse:44735.75939
[32]	validation_0-rmse:43896.50740
[33]	validation_0-rmse:42960.82986
[34]	validation_0-rmse:42165.62298
[35]	validation_0-rmse:41449.70513
[36]	validation_0-rmse:40841.96505
[37]	validation_0-rmse:40135.18677
[38]	validation_0-rmse:39546.06681
[39]	validation_0-rmse:39013.40152
[40]	validation_0-rmse:38508.41165
[41]	validation_0-rmse:38066.95715
[42]	validation_0-rmse:37615.52224
[43]	validation_0-rm

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[16]	validation_0-rmse:68111.82137
[17]	validation_0-rmse:65657.95518
[18]	validation_0-rmse:63349.58589
[19]	validation_0-rmse:61167.24591
[20]	validation_0-rmse:59333.41679
[21]	validation_0-rmse:57404.70248
[22]	validation_0-rmse:55793.09132
[23]	validation_0-rmse:54072.88074
[24]	validation_0-rmse:52636.07133
[25]	validation_0-rmse:51113.69582
[26]	validation_0-rmse:49617.28166
[27]	validation_0-rmse:48435.51412
[28]	validation_0-rmse:47135.73125
[29]	validation_0-rmse:46134.55118
[30]	validation_0-rmse:45004.36903
[31]	validation_0-rmse:44121.65184
[32]	validation_0-rmse:43340.71990
[33]	validation_0-rmse:42428.56869
[34]	validation_0-rmse:41543.44996
[35]	validation_0-rmse:40895.02591
[36]	validation_0-rmse:40256.23446
[37]	validation_0-rmse:39533.89730
[38]	validation_0-rmse:39014.40773
[39]	validation_0-rmse:38545.16273
[40]	validation_0-rmse:38101.21711
[41]	validation_0-rmse:37684.04649
[42]	validation_0-rmse:37189.93659
[43]	validation_0-rmse:36840.22573
[44]	validation_0-rm

...

[17]	validation_0-rmse:64458.43568
[18]	validation_0-rmse:61926.03220
[19]	validation_0-rmse:59595.47237
[20]	validation_0-rmse:57630.68104
[21]	validation_0-rmse:55556.36761
[22]	validation_0-rmse:53797.17909
[23]	validation_0-rmse:51928.14489
[24]	validation_0-rmse:50419.92938
[25]	validation_0-rmse:48759.13020
[26]	validation_0-rmse:47173.75826
[27]	validation_0-rmse:45882.87267
[28]	validation_0-rmse:44597.74823
[29]	validation_0-rmse:43522.82148
[30]	validation_0-rmse:42294.15955
[31]	validation_0-rmse:41368.30026
[32]	validation_0-rmse:40466.02136
[33]	validation_0-rmse:39495.46170
[34]	validation_0-rmse:38661.58034
[35]	validation_0-rmse:37947.70000
[36]	validation_0-rmse:37267.49482
[37]	validation_0-rmse:36550.42486
[38]	validation_0-rmse:36008.16192
[39]	validation_0-rmse:35490.80922
[40]	validation_0-rmse:35014.57780
[41]	validation_0-rmse:34541.34719
[42]	validation_0-rmse:34113.17815
[43]	validation_0-rmse:33752.11500
[44]	validation_0-rmse:33294.43396
[45]	validation_0-rm

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[15]	validation_0-rmse:68974.06733
[16]	validation_0-rmse:66277.01553
[17]	validation_0-rmse:63768.46083
[18]	validation_0-rmse:61357.04651
[19]	validation_0-rmse:59095.75707
[20]	validation_0-rmse:57185.91684
[21]	validation_0-rmse:55194.11735
[22]	validation_0-rmse:53531.05739
[23]	validation_0-rmse:51782.48778
[24]	validation_0-rmse:50285.43621
[25]	validation_0-rmse:48705.55662
[26]	validation_0-rmse:47216.30122
[27]	validation_0-rmse:46010.36851
[28]	validation_0-rmse:44678.36991
[29]	validation_0-rmse:43647.34513
[30]	validation_0-rmse:42484.81099
[31]	validation_0-rmse:41590.56849
[32]	validation_0-rmse:40784.46031
[33]	validation_0-rmse:39818.75600
[34]	validation_0-rmse:38925.55690
[35]	validation_0-rmse:38237.04039
[36]	validation_0-rmse:37605.05610
[37]	validation_0-rmse:36866.49264
[38]	validation_0-rmse:36318.82674
[39]	validation_0-rmse:35803.64634
[40]	validation_0-rmse:35331.06345
[41]	validation_0-rmse:34939.86599
[42]	validation_0-rmse:34538.33394
[43]	validation_0-rm

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[16]	validation_0-rmse:68741.14152
[17]	validation_0-rmse:66153.81645
[18]	validation_0-rmse:63635.41022
[19]	validation_0-rmse:61331.09193
[20]	validation_0-rmse:59355.72554
[21]	validation_0-rmse:57238.31478
[22]	validation_0-rmse:55561.14528
[23]	validation_0-rmse:53805.91868
[24]	validation_0-rmse:52349.38028
[25]	validation_0-rmse:50665.86029
[26]	validation_0-rmse:49080.53372
[27]	validation_0-rmse:47851.18199
[28]	validation_0-rmse:46563.47607
[29]	validation_0-rmse:45545.24913
[30]	validation_0-rmse:44292.70683
[31]	validation_0-rmse:43386.75080
[32]	validation_0-rmse:42567.92966
[33]	validation_0-rmse:41487.28463
[34]	validation_0-rmse:40520.53585
[35]	validation_0-rmse:39861.68287
[36]	validation_0-rmse:39253.46128
[37]	validation_0-rmse:38454.30200
[38]	validation_0-rmse:37938.12653
[39]	validation_0-rmse:37326.81671
[40]	validation_0-rmse:36875.96716
[41]	validation_0-rmse:36445.45771
[42]	validation_0-rmse:36056.61122
[43]	validation_0-rmse:35675.55344
[44]	validation_0-rm

In [8]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('XGB_cv_result.csv',index=False)